In [1]:
import numpy as np
import pandas as pd

In [54]:
customer = pd.read_csv("./olist_customers_dataset.csv")
location=pd.read_csv("./olist_geolocation_dataset.csv")
order_item=pd.read_csv("./olist_order_items_dataset.csv")
order=pd.read_csv("./olist_orders_dataset.csv")
review=pd.read_csv("./olist_order_reviews_dataset.csv")
sellers=pd.read_csv("./olist_sellers_dataset.csv")
product=pd.read_csv("./olist_products_dataset.csv")

In [30]:
# Step 1: orders + customers
order_customer = pd.merge(order, customer, on="customer_id", how="left")

# Step 2: add reviews (order_id)
#order_customer_review = pd.merge(order_customer, review, on="order_id", how="left")

# Step 3: add order_items (order_id)
order_customer_item = pd.merge(order_customer, order_item, on="order_id", how="left")

# Group by zip code and take the average of lat/lng
clean_geo = (
    location.groupby(["geolocation_zip_code_prefix", "geolocation_state"], as_index=False)
       .agg({
           "geolocation_lat": "mean",
           "geolocation_lng": "mean"
       })
)
# Step 4: add geolocation (via zip_code_prefix of customers) 
order_customer_item_location = pd.merge(order_customer_review_item, clean_geo, left_on="customer_zip_code_prefix", right_on="geolocation_zip_code_prefix", how="left")

In [31]:
#checking duplicates
clean_geo['geolocation_zip_code_prefix'].value_counts().sort_values(ascending = False)

geolocation_zip_code_prefix
21550    2
4011     2
23056    2
2116     2
72915    2
        ..
1019     1
1020     1
1021     1
1022     1
99880    1
Name: count, Length: 19015, dtype: int64

In [35]:
clean_geo = clean_geo.drop_duplicates(subset=["geolocation_zip_code_prefix"]) #cleaimg duplicates
clean_geo['geolocation_zip_code_prefix'].value_counts().sort_values(ascending = False) #checking if all duplicates r gone

geolocation_zip_code_prefix
1001     1
1002     1
1003     1
1004     1
1005     1
        ..
99960    1
99965    1
99970    1
99980    1
99990    1
Name: count, Length: 19015, dtype: int64

In [33]:
order_customer_item_location.shape

(114188, 28)

In [39]:
order_customer_item_location.to_csv("shipping.csv",index=False)

In [ ]:
# recreating new csv file after finding that i didnt add seller loc

In [55]:

# --- Step 1: Orders + Customers ---
order_customer = pd.merge(order, customer, on="customer_id", how="left")

# --- Step 2: Add Order Items ---
order_customer_item = pd.merge(order_customer, order_item, on="order_id", how="left")

# --- Step 3: Add Sellers ---
order_customer_item_seller = pd.merge(order_customer_item, sellers, on="seller_id", how="left")

# --# --- Step 4: Add products ---
order_customer_item_seller_product = pd.merge(order_customer_item_seller, product, on="product_id", how="left")
# --- Step 5: Clean Geolocation ---
clean_geo = (
    location.groupby(["geolocation_zip_code_prefix", "geolocation_state"], as_index=False)
       .agg({
           "geolocation_lat": "mean",
           "geolocation_lng": "mean"
       })
)
clean_geo = clean_geo.drop_duplicates(subset=["geolocation_zip_code_prefix"]) #cleaning duplicates

# --- Step 6: Add Customer Geolocation ---
order_customer_item_seller_product_geo = pd.merge(
    order_customer_item_seller_product,
    clean_geo,
    left_on="customer_zip_code_prefix",
    right_on="geolocation_zip_code_prefix",
    how="left",
    suffixes=("", "_cust")
)

# --- Step 7: Add Seller Geolocation ---
order_customer_item_seller_product_geo = pd.merge(
    order_customer_item_seller_product_geo,
    clean_geo,
    left_on="seller_zip_code_prefix",
    right_on="geolocation_zip_code_prefix",
    how="left",
    suffixes=("_cust", "_sell")
)

# --- Step 8: Drop unnecessary duplicate columns ---
order_customer_item_seller_product_geo.drop(columns=["geolocation_zip_code_prefix_cust", "geolocation_zip_code_prefix_sell"], inplace=True)


In [56]:
order_customer_item_seller_product_geo.shape

(113425, 35)

In [58]:
order_customer_item_seller_product_geo.to_csv("shipping_pupdated.csv",index=False)